# 1.1 Particle picking

In [ ]:
from aitom.bin.picking import picking
from particle_picking_and_autoencoder_util import mkdir,ParticlePicking,ImageSliceViewer3D

# file path
path = '/ldap_shared/home/v_zhenxi_zhu/data/aitom_demo_single_particle_tomogram.mrc'
output_dir = './tmp/picking'
mkdir(output_dir)
pick = ParticlePicking(path,output_dir)

# select sigma1 automatically 
sigma1 = pick.select_sigma()

# or select it manually
if True:
    sigma1 = 5

# particle picking
result = picking(path, s1=sigma1, s2=sigma1*1.1, t=3, find_maxima=False, partition_op=None, multiprocessing_process_num=10, pick_num=1000)
print("DoG done, %d particles picked" % len(result))

# Save subvolumes of peaks for autoencoder input
pick.dump_subvol(result)

# 1.2 Visualization of particle picking result

In [ ]:
%%html
<style>
div.ui-dialog-titlebar {display: none;}
</style>
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# Set display style

In [ ]:
# Visualization of the tomogram and subtomograms
view_tomogram = True
if view_tomogram:
    # Visualization of the tomogram
    %matplotlib notebook
    ImageSliceViewer3D(path, output_dir)
    # pick.view_tomo()
else:
    # Visualization of the subtomograms
    subvol_num = 113
    pick.view_subtom(subvol_num)

# 1.3 Manual selection

In [ ]:
# manual remove particles
remove_particles = [0,4,9,11,18,24,26,30,34,35,38,53,66,78,96,99,111,112,118,123,124]
pick_num = 100
pick.select(remove_particles,pick_num)

# 2.1 Autoencoder (single particle)

In [ ]:
from particle_picking_and_autoencoder_util import mkdir,view_clusters
import aitom.classify.deep.unsupervised.autoencoder.autoencoder as AE
import aitom.io.file as AIF

subvols_loc = './tmp/picking/selected_demo_single_particle_subvolumes.pickle'
output_dir = './tmp/autoencoder_single_particle'
mkdir(output_dir)

# pickle data file of CECT small subvolumes
d = AIF.pickle_load(subvols_loc)
AE.encoder_simple_conv_test(d=d, pose=None, img_org_file=False, out_dir=output_dir, clus_num=1)
AE.kmeans_centers_plot(AE.op_join(output_dir, 'clus-center'))

## Visualization of cluster centers

In [ ]:
view_clusters(output_dir)

# 2.2 Autoencoder (multi particles)

In [ ]:
# generate subvolumes from pickle file
from particle_picking_and_autoencoder_util import combine_subtom
out_dir = './tmp/picking/'
# demo subtomograms, can download from: https://cmu.box.com/s/9hn3qqtqmivauus3kgtasg5uzlj53wxp
pickle_path = '/ldap_shared/home/v_zhenxi_zhu/classification_and_averaging/aitom_demo_subtomograms.pickle' 
# mix particle picking results and demo subtomograms to test the classification task
combine_subtom(out_dir,pickle_path)

In [ ]:
from particle_picking_and_autoencoder_util import mkdir,view_clusters
import aitom.classify.deep.unsupervised.autoencoder.autoencoder as AE
import aitom.io.file as AIF

subvols_loc = './tmp/picking/subvolumes.pickle'
output_dir = './tmp/autoencoder_multi_particles'
mkdir(output_dir)

# pickle data file of CECT small subvolumes
d = AIF.pickle_load(subvols_loc) # pickle data file of CECT small subvolumes
AE.encoder_simple_conv_test(d=d, pose=None, img_org_file=False, out_dir=output_dir, clus_num=3)
AE.kmeans_centers_plot(AE.op_join(output_dir, 'clus-center'))

## Visualization of cluster centers

In [ ]:
view_clusters(output_dir)